# TV Tropes
## The stories underlying stories


In [15]:
import numpy as np
#import itertools
import matplotlib.pyplot as plt
import IPython.display as ipd
#from ipywidgets import interact
import re

#import sklearn.datasets as datasets
#import pandas as pd
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.externals.six import StringIO  
#from IPython.display import Image
#from IPython.display import display
#from sklearn.tree import export_graphviz
#import pydotplus

import json
import functools as ft

import sys

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)

### First, we grab data

## Original source
https://old.datahub.io/dataset/dbtropes

This is an rdf database. Since we are just looking at what movies have which tropes, we can us the database created here: https://arxiv.org/pdf/1809.10959.pdf. 
This has been imported into this project as all_films_and_their_tropes.json.


In [7]:
with open("all_films_and_their_tropes.json",'r') as f:
    data = json.loads(f.read())

The form of 'data' is a dictionary from movie names to the names of their associated tropes. 

In [34]:
print(len(data))
print(list(data.keys())[0:10])
print(data['ABeautifulMind'])
all_tropes = list(ft.reduce(lambda x,y: x.union(set(y)), data.values(), set()))
all_movies = list(data.keys())

print()
print(all_tropes[0])
print(len(all_tropes))

5925
['ABeautifulMind', 'ABetterTomorrow', 'ABirdersGuideToEverything', 'ABittersweetLife', 'ABoyAndHisDog', 'ABridgeTooFar', 'ABronxTale', 'ACaseOfSpringFever', 'AChristmasCarol2009', 'AChristmasStory']
['Foreshadowing', 'RuleOfPerception', 'AwardBaitSong', 'SoundtrackDissonance', 'PragmaticAdaptation', 'OffScreenTeleportation', 'ForegoneConclusion', 'ImaginaryFriend', 'OscarBait', 'TheReveal', 'IvyLeagueForEveryone', 'TakeAThirdOption', 'Hallucinations', 'AbsentMindedProfessor', 'TheLoinsSleepTonight', 'HollywoodNerd', 'ShoutOut', 'DoYouWantToCopulate', 'BrokenAce', 'TheBigBoard', 'SheIsAllGrownUp', 'PoliticallyCorrectHistory', 'AllThereIsToKnowAboutTheCryingGame', 'NotThatKindOfDoctor', 'RedScare', 'BunnyEarsLawyer', 'UnreliableNarrator', 'RoomFullOfCrazy', 'HollywoodScience', 'AdultFear', 'ForgetsToEat', 'SmartPeoplePlayChess', 'MeaningfulEcho', 'GoodWithNumbers', 'MentalStory', 'AnswerCut', 'NoMedicationForMe', 'EEqualsMCHammer', 'WritersCannotDoMath', 'EurekaMoment', 'EveryoneLov

Notice that there are enough features that the data should not be represented as a (non-sparse) matrix. Instead, keep as a dictionary.

## Clustering

In [40]:
#from sklearn.feature_extraction import DictVectorizer
#from sklearn.cluster import KMeans
import scipy.sparse as sp

trope_to_num_dict = {v:i for i,v in enumerate(all_tropes)}
def tropeToNum(v): 
    return trope_to_num_dict[v]

movies_to_tropes = sp.dok_matrix((len(all_movies),len(all_tropes)), dtype=np.int8)

for m in range(len(all_movies)):
    tropes = data[all_movies[m]]
    for i in map(tropeToNum, tropes):
        movies_to_tropes[m, i] = 1
    
#movies_to_tropes = movies_to_tropes.transpose().tocsr()
print(movies_to_tropes.shape)

(5925, 18270)


Now that we have a sparse representation, we still want to do a dimensionality reduction so that we can do clustering.